In [69]:
from copy import deepcopy
import random

class myPlayer(object):
    
    def __init__(self, color=None):
        self.color=color #1 black 2 white
        self.opponent=None
        self.prev_state = []
        self.curr_state=[]
        self.visit=[[0 for i in range(5)] for j in range(5)]
        self.suicide_move=0
        
    def read_file(self, file):
        with open(file, 'r') as f:
            list = f.readlines()
            for i in range(len(list)):
                line=list[i].rstrip('\n')
                if i==0:
                    self.color=int(line)
                    self.opponent=3-self.color
                if i in [1,2,3,4,5]:
                    self.prev_state.append([int(i) for i in line])
                if i in [6,7,8,9,10]:
                    self.curr_state.append([int(i) for i in line])
    
    def output(self, move):
        f = open('output.txt','w+')
        if move == 'PASS':
            f.write(move)
        else:
            f.write(str(move[0])+','+str(move[1]))
    
    def evaluate(self, board, player):
        max_val=0
        min_val=0
        eva_max=0
        eva_min=0
        for i in range(5):
            for j in range(5):
                if board[i][j]==self.color:
                    eva_max+=1
                    max_val+= eva_max+ self.is_alive(i, j, board, player)
                    
                elif board[i][j]==3-self.color:
                    eva_min+=1
                    min_val+= eva_min+ self.is_alive(i, j, board, player)

        if player==self.color:
            return max_val-min_val
        return min_val-max_val 

    def clear_visit(self):
        self.visit =[[0 for i in range(5)] for j in range(5)]
        self.count=0
        
    def dfs(self, i, j, board, player): #find liberty
        self.visit[i][j]=1
        directions = [[i - 1, j], [i + 1, j], [i, j - 1], [i, j + 1]]
        for dx, dy in directions:
            if dx<0 or dx>4 or dy<0 or dy>4:
                continue
            elif self.visit[dx][dy]==0:
                if board[dx][dy]==0:
                    self.count+=1
                    #return 1
                elif board[dx][dy]==board[i][j]:
                    self.dfs(dx, dy, board, player)
                else:
                    continue

        return self.count
    
    def is_alive(self, x, y, board, player):
        self.clear_visit()
        isalive =self.dfs(x,y,board,player)            
        return isalive
    
    def dead_stones(self, board, player):
        dead =[]
        for i in range(5):
            for j in range(5):
                if board[i][j]==player:
                    if not self.is_alive(i, j, board, player) and (i ,j) not in dead:
                        dead.append((i,j))
                        
        return dead
    
    def capture(self, board, player):
        opponent=3-player
   
        capture_list= self.dead_stones(board,player)
        if capture_list:
            for i,j in capture_list:
                board[i][j]=0
        
        for i in range(5):
            for j in range(5):
                if board[i][j]==player:
                    if self.is_alive(i,j,board, player)==0:
                        self.suicide_move=1
                        break
        
        return board
    
    def valid_move_check(self, i, j, player, prev_board, board):
        
        if not (i >= 0 and i < len(board)):
            return False
        if not (j >= 0 and j < len(board)):
             return False
        
        if board[i][j] != 0:
             return False
        
        test_board = deepcopy(board)
        prev = deepcopy(prev_board)

        test_board[i][j] = player
        dead_pieces=self.dead_stones(test_board, 3-player)
        if self.is_alive(i, j, test_board, player)>0:
            return True
        
        self.capture(test_board, 3-player)
        if not self.is_alive(i, j, test_board, player):
            return False

        else:
            if dead_pieces and self.KO(prev, test_board):
                return False
        return True
    
    def KO(self, prev, curr):
        for i in range(5):
            for j in range(5):
                if prev[i][j]!=curr[i][j]:
                    return False
        return True
 

    def minimax(self, prev_board, curr_board, eva, depth, player, alpha, beta):
        if depth == 0:
            return eva
        best = eva
        new_prev =deepcopy(curr_board)

        for i in range(5):
            for j in range(5):
                if self.valid_move_check(i, j, self.color, prev_board, curr_board):
                    new_board=deepcopy(curr_board)
                    new_board[i][j]=player
                    new_board=self.capture(new_board, 3-player)

                    if self.suicide_move==1:
                        self.suicide_move=0
                        continue
                    else:

                        eva = self.evaluate(new_board, 3-player)
                        next_eva= self.minimax(new_prev, new_board, eva, 
                                               depth-1, 3-player, alpha, beta)

                        curr_score = -1 * next_eva
       
                        if curr_score > best:
                            best = curr_score
     
                        new_score = -1 * best

                        if player == 3-self.color:
                            score = new_score
                            if score < alpha:
                                return best
                            if best > beta:
                                beta = best
 
                        elif player == self.color:
                            score = new_score
                            if score < beta:
                                return best
                            if best > alpha:
                                alpha = best

        return best

    def findBestMove(self, depth, alpha, beta):
        board =deepcopy(self.curr_state)
        prev =deepcopy(self.prev_state)
        best=0
        moves=[]
        for i in range(5):
            for j in range(5):
                if self.valid_move_check(i, j, self.color, prev, board) :
                    test_board = deepcopy(board)
                    test_board[i][j] = self.color
                    test_board = self.capture(test_board, 3-self.color)
      
                    if self.suicide_move==1:
                        self.suicide_move=0
                        continue
                    else:
                        eva = self.evaluate(test_board, 3-self.color)
                        moveVal = self.minimax(board, test_board, eva, depth,
                                               3-self.color, alpha, beta)
                        curr_score = -1 * moveVal

                        if curr_score > best or not moves:
                            best = curr_score
                            alpha = best
                            moves = [(i,j)]

                        elif curr_score == best:
                            moves.append((i,j))

        return moves

    def play(self):
        self.read_file('input.txt')
        start=0
        start_bool=False
        for i in range(5):
            for j in range(5):
                if self.curr_state[i][j]!=0:
                    if i==2 and j==2:
                        start_bool = True
                    start+=1

        if (start==0 and self.color==1) or (start==1 and self.color==2 and start_bool is False):
            action = [(2,2)]
        else:
            action = self.findBestMove( 2, -1000, -1000)

        if action == []:
            rand_action = ['PASS']
        else:
            rand_action = random.choice(action)
        
        self.output(rand_action)

p = myPlayer()
p.play()



In [67]:
from copy import deepcopy
import random

class myPlayer(object):
    
    def __init__(self, color=None):
        self.color=color #1 black 2 white
        self.opponent=None
        self.prev_state = []
        self.curr_state=[]
        self.visit=[[0 for i in range(5)] for j in range(5)]
        self.suicide_move=0
        
    def read_file(self, file):
        with open(file, 'r') as f:
            list = f.readlines()
            for i in range(len(list)):
                line=list[i].rstrip('\n')
                if i==0:
                    self.color=int(line)
                    self.opponent=3-self.color
                if i in [1,2,3,4,5]:
                    self.prev_state.append([int(i) for i in line])
                if i in [6,7,8,9,10]:
                    self.curr_state.append([int(i) for i in line])
    
    def output(self, move):
        f = open('output.txt','w+')
        if move == 'PASS':
            f.write(move)
        else:
            f.write(str(move[0])+','+str(move[1]))
    
    def evaluate(self, board, player):
        max_val=0
        min_val=0
        eva_max=0
        eva_min=0
        for i in range(5):
            for j in range(5):
                if board[i][j]==self.color:
                    eva_max+=1
                    max_val+= eva_max+ self.is_alive(i, j, board, player)
                    
                elif board[i][j]==3-self.color:
                    eva_min+=1
                    min_val+= eva_min+ self.is_alive(i, j, board, player)

        if player==self.color:
            return max_val-min_val
        return min_val-max_val 

    def clear_visit(self):
        self.visit =[[0 for i in range(5)] for j in range(5)]
        self.count=0
        
    def dfs(self, i, j, board, player): #find liberty
        self.visit[i][j]=1
        directions = [[i - 1, j], [i + 1, j], [i, j - 1], [i, j + 1]]
        for dx, dy in directions:
            if dx<0 or dx>4 or dy<0 or dy>4:
                continue
            elif self.visit[dx][dy]==0:
                if board[dx][dy]==0:
                    self.count+=1
                    #return 1
                elif board[dx][dy]==board[i][j]:
                    self.dfs(dx, dy, board, player)
                else:
                    continue

        return self.count
    #1有气 0无气
    
    def is_alive(self, x, y, board, player):
        self.clear_visit()
        isalive =self.dfs(x,y,board,player)            
        return isalive
    
    def dead_stones(self, board, player):
        dead =[]
        for i in range(5):
            for j in range(5):
                if board[i][j]==player:
                    if not self.is_alive(i, j, board, player) and (i ,j) not in dead:
                        dead.append((i,j))
                        
        return dead
    
    def capture(self, board, player):
        opponent=3-player
   
        capture_list= self.dead_stones(board,player)
        if capture_list:
            for i,j in capture_list:
                board[i][j]=0
        
        for i in range(5):
            for j in range(5):
                if board[i][j]==player:
                    if self.is_alive(i,j,board, player)==0:
                        self.suicide_move=1
                        break
        
        return board
    
    def valid_move_check(self, i, j, player, prev_board, board):
        
        # Check if the place is in the board range
        if not (i >= 0 and i < len(board)):
            return False
        if not (j >= 0 and j < len(board)):
             return False
        
        # Check if the place already has a piece
        if board[i][j] != 0:
             return False
        
        test_board = deepcopy(board)
        prev = deepcopy(prev_board)
        # Check if the place has liberty
        test_board[i][j] = player
        dead_pieces=self.dead_stones(test_board, 3-player)
        if self.is_alive(i, j, test_board, player)>0:
            return True

        # If not, remove the died pieces of opponent and check again
        self.capture(test_board, 3-player)
        if not self.is_alive(i, j, test_board, player):
            return False

        # Check special case: repeat placement causing the repeat board state (KO rule)
        else:
            if dead_pieces and self.KO(prev, test_board):
                return False
        return True
    
    def KO(self, prev, curr):
        for i in range(5):
            for j in range(5):
                if prev[i][j]!=curr[i][j]:
                    return False
        return True
 

    def minimax(self, prev_board, curr_board, eva, depth, player, alpha, beta):
        if depth == 0:
            return eva
        best = eva
        new_prev =deepcopy(curr_board)

        for i in range(5):
            for j in range(5):
                if self.valid_move_check(i, j, self.color, prev_board, curr_board):
                    new_board=deepcopy(curr_board)
                    new_board[i][j]=player
                    new_board=self.capture(new_board, 3-player)

                    if self.suicide_move==1:
                        self.suicide_move=0
                        continue
                    else:

                        eva = self.evaluate(new_board, 3-player)
                        next_eva= self.minimax(new_prev, new_board, eva, 
                                               depth-1, 3-player, alpha, beta)

                        curr_score = -1 * next_eva
                        # check if new result is better than current best
                        if curr_score > best:
                            best = curr_score
                        # update the score
                        new_score = -1 * best

                        # Alpha beta pruning
                        # if we are looking at the minimizing player (opponent)
                        if player == 3-self.color:
                            score = new_score
                            # check if prune
                            if score < alpha:
                                return best
                            # if we dont prune, update beta
                            if best > beta:
                                beta = best
                        # if we are looking at the maximizing player (ourselves)
                        elif player == self.color:
                            score = new_score
                            # check prune
                            if score < beta:
                                return best
                            # if we dont prune, update alpha
                            if best > alpha:
                                alpha = best

        return best

    def findBestMove(self, depth, alpha, beta):
        board =deepcopy(self.curr_state)
        prev =deepcopy(self.prev_state)
        best=0
        moves=[]
        for i in range(5):
            for j in range(5):
                #valid_move_check(self, i, j, player, prev_board, board)
                if self.valid_move_check(i, j, self.color, prev, board) :
                    #print(i,j)
                    test_board = deepcopy(board)
                    # Make the move
                    test_board[i][j] = self.color
                    test_board = self.capture(test_board, 3-self.color)
                    #print(test_board)
                    # compute evaluation function for this
                    # move.
                    if self.suicide_move==1:
                        self.suicide_move=0
                        continue
                    else:
                        eva = self.evaluate(test_board, 3-self.color)
                        #print('eva:'+str(eva))
                        #eva, depth, player, alpha, beta
                        moveVal = self.minimax(board, test_board, eva, depth,
                                               3-self.color, alpha, beta)
                        curr_score = -1 * moveVal
                        #print('curr_score:'+str(curr_score))
                        # check if moves is empty or if we have new best move(s)
                        if curr_score > best or not moves:
                            best = curr_score
                            alpha = best
                            moves = [(i,j)]
                        # if we have another best move then we add to the moves list
                        elif curr_score == best:
                            moves.append((i,j))

        return moves

    def play(self):
        self.read_file('input.txt')
        start=0
        start_bool=False
        for i in range(5):
            for j in range(5):
                if self.curr_state[i][j]!=0:
                    if i==2 and j==2:
                        start_bool = True
                    start+=1

        if (start==0 and self.color==1) or (start==1 and self.color==2 and start_bool is False):
            action = [(2,2)]
        else:
            action = self.findBestMove( 2, -1000, -1000)
            #print(action)
        # if empty list, then no action, choose to pass
        if action == []:
            rand_action = ['PASS']
        # else choose a random action from the list
        else:
            rand_action = random.choice(action)
        
        print(rand_action)
        self.output(rand_action)

p = myPlayer()
p.play()

(3, 2)


In [63]:
from copy import deepcopy
import random

class myPlayer(object):
    
    def __init__(self, color=None):
        self.color=color #1 black 2 white
        self.opponent=None
        self.prev_state = []
        self.curr_state=[]
        self.visit=[[0 for i in range(5)] for j in range(5)]
        self.suicide_move=0
        
    def read_file(self, file):
        with open(file, 'r') as f:
            list = f.readlines()
            for i in range(len(list)):
                line=list[i].rstrip('\n')
                if i==0:
                    self.color=int(line)
                    self.opponent=3-self.color
                if i in [1,2,3,4,5]:
                    self.prev_state.append([int(i) for i in line])
                if i in [6,7,8,9,10]:
                    self.curr_state.append([int(i) for i in line])
    
    def output(self, move):
        f = open('output.txt','w+')
        if move == 'PASS':
            f.write(move)
        else:
            f.write(str(move[0])+','+str(move[1]))
    
    def evaluate(self, board, player):
        max_val=0
        min_val=0
        eva_max=0
        eva_min=0
        for i in range(5):
            for j in range(5):
                if board[i][j]==self.color:
                    eva_max+=1
                    max_val+= eva_max+ self.is_alive(i, j, board, player)
                    
                elif board[i][j]==3-self.color:
                    eva_min+=1
                    min_val+= eva_min+ self.is_alive(i, j, board, player)

        if player==self.color:
            return max_val-min_val
        return min_val-max_val 
    '''
    def evaluate(self, board, np):
        player, opponent, heur_player, heur_opponent = 0, 0, 0, 0
        for i in range(5):
            for j in range(5):
                if board[i][j] == color:
                    player += 1
                    #print(str((i,j))+"alive:"+str(self.is_alive(i, j, board, player)))
                    heur_player += (player + self.is_alive(i, j, board, player))
                elif board[i][j] == 3 - color:
                    opponent += 1
                    #print(str((i,j))+"alive:"+str(self.is_alive(i, j, board, player)))
                    heur_opponent += (opponent + self.is_alive(i, j, board, player))

        if np == color:
            return heur_player - heur_opponent
        return heur_opponent - heur_player
    '''
    def clear_visit(self):
        self.visit =[[0 for i in range(5)] for j in range(5)]
        self.count=0
        
    def dfs(self, i, j, board, player): #find liberty
        self.visit[i][j]=1
        directions = [[i - 1, j], [i + 1, j], [i, j - 1], [i, j + 1]]
        for dx, dy in directions:
            if dx<0 or dx>4 or dy<0 or dy>4:
                continue
            elif self.visit[dx][dy]==0:
                if board[dx][dy]==0:
                    self.count+=1
                    #return 1
                elif board[dx][dy]==board[i][j]:
                    self.dfs(dx, dy, board, player)
                else:
                    continue

        return self.count
    #1有气 0无气
    
    def is_alive(self, x, y, board, player):
        self.clear_visit()
        isalive =self.dfs(x,y,board,player)            
        return isalive
    
    def dead_stones(self, board, player):
        dead =[]
        for i in range(5):
            for j in range(5):
                if board[i][j]==player:
                    if not self.is_alive(i, j, board, player) and (i ,j) not in dead:
                        dead.append((i,j))
                        
        return dead
    
    def capture(self, board, player):
        opponent=3-player
        '''
        capture_list=[]
        
        for i in range(5):
            for j in range(5):
                if board[i][j]==0:
                    continue
                elif board[i][j]==opponent and self.is_alive(i, j, board, opponent)==0:
                    capture_list.append([i,j])
        '''
        capture_list= self.dead_stones(board,player)
        if capture_list:
            for i,j in capture_list:
                board[i][j]=0
        '''
        for i in range(5):
            for j in range(5):
                if board[i][j]==player:
                    if self.is_alive(i,j,board, player)==0:
                        self.suicide_move=1
                        break
        '''
        return board
    
    def valid_move_check(self, i, j, player, prev_board, board):
        
        # Check if the place is in the board range
        if not (i >= 0 and i < len(board)):
            return False
        if not (j >= 0 and j < len(board)):
             return False
        
        # Check if the place already has a piece
        if board[i][j] != 0:
             return False
        
        test_board = deepcopy(board)
        prev = deepcopy(prev_board)
        # Check if the place has liberty
        test_board[i][j] = player
        dead_pieces=self.dead_stones(test_board, 3-player)
        if self.is_alive(i, j, test_board, player)>0:
            return True

        # If not, remove the died pieces of opponent and check again
        self.capture(test_board, 3-player)
        if not self.is_alive(i, j, test_board, player):
            return False

        # Check special case: repeat placement causing the repeat board state (KO rule)
        else:
            if dead_pieces and self.KO(prev, test_board):
                return False
        return True
    
    def KO(self, prev, curr):
        for i in range(5):
            for j in range(5):
                if prev[i][j]!=curr[i][j]:
                    return False
        return True
 
    '''    
    def KO(self, i, j, board, player):
        new_board=capture(i,j,board,player)
        if new_board==prev_board:
            return True
        else:
            return False
    '''
        
        
    ''' 
    def game_end(self, board, passs):
        n_move=0
        for i in board:
            for j in i:
                if j != 0:
                    n_move+=1
        # Case 1: max move reached
        if n_move>=24:
            return True
        # Case 2: two players all pass the move.
        if passs==2:
            return True
        return False
    '''

    def minimax(self, prev_board, curr_board, eva, depth, player, alpha, beta):
        if depth == 0:
            return eva
        best = eva
        new_prev =deepcopy(curr_board)

        for i in range(5):
            for j in range(5):
                if self.valid_move_check(i, j, self.color, prev_board, curr_board):
                    new_board=deepcopy(curr_board)
                    new_board[i][j]=player
                    new_board=self.capture(new_board, 3-player)

                    eva = self.evaluate(new_board, 3-player)
                    next_eva= self.minimax(new_prev, new_board, eva, 
                                           depth-1, 3-player, alpha, beta)

                    curr_score = -1 * next_eva
                    # check if new result is better than current best
                    if curr_score > best:
                        best = curr_score
                    # update the score
                    new_score = -1 * best

                    # Alpha beta pruning
                    # if we are looking at the minimizing player (opponent)
                    if player == 3-self.color:
                        score = new_score
                        # check if prune
                        if score < alpha:
                            return best
                        # if we dont prune, update beta
                        if best > beta:
                            beta = best
                    # if we are looking at the maximizing player (ourselves)
                    elif player == self.color:
                        score = new_score
                        # check prune
                        if score < beta:
                            return best
                        # if we dont prune, update alpha
                        if best > alpha:
                            alpha = best

        return best

    '''
        score = self.evaluate(board, self.color)
        if self.game_end(board, passs) or depth==6:
            return score

        if(isMax):
            best =-1000
            valid_moves=0
            for i in range(5):
                for j in range(5):
                    if self.valid_move_check(i, j, self.color, prev_board, board):
                        test_board=deepcopy(board)
                        valid_moves+=1
                        test_board[i][j]=self.color
                        test_board =self.capture(test_board, self.color)
        
                        if self.suicide_move==1:
                            self.suicide_move=0
                            best = best
                        else:
                            best = max(best, self.minimax(new_prev, test_board, 
                                                          depth+1, not isMax, passs, alpha, beta))
                            alpha=max(alpha, best)
                            if beta<=alpha:
                                break
                            board[i][j]=0
            if valid_moves==0:
                passs+=1
                best = max(best, self.minimax(new_prev, board, 
                                              depth+1, not isMax, passs, alpha, beta))
            return best

        else:
            best = 1000
            valid_moves=0
            for i in range(5):
                for j in range(5):
                    if self.valid_move_check(i, j, self.opponent, prev_board, board):
                        test_board=deepcopy(board)
                        valid_moves+=1
                        test_board[i][j]=self.opponent
                        test_board =self.capture(test_board, self.opponent)
                    
                        if self.suicide_move==1:
                            self.suicide_move=0
                            best = best
                        else:
                            best = min(best, self.minimax(new_prev, test_board, 
                                                          depth+1, not isMax, passs, alpha, beta))
                            beta=min(beta, best)
                            if beta<=alpha:
                                break
                            board[i][j]=0
            if valid_moves==0:
                passs+=1
                best = min(best, self.minimax(new_prev, board, 
                                              depth+1, not isMax, passs, alpha, beta))
            return best
    '''  
    def findBestMove(self, depth, alpha, beta):
        board =deepcopy(self.curr_state)
        prev =deepcopy(self.prev_state)
        best=0
        moves=[]
        for i in range(5):
            for j in range(5):
                #valid_move_check(self, i, j, player, prev_board, board)
                if self.valid_move_check(i, j, self.color, prev, board) :
                    #print(i,j)
                    test_board = deepcopy(board)
                    # Make the move
                    test_board[i][j] = self.color
                    test_board = self.capture(test_board, 3-self.color)
                    #print(test_board)
                    # compute evaluation function for this
                    # move.
                    eva = self.evaluate(test_board, 3-self.color)
                    #print('eva:'+str(eva))
                    #eva, depth, player, alpha, beta
                    moveVal = self.minimax(board, test_board, eva, depth,
                                           3-self.color, alpha, beta)
                    curr_score = -1 * moveVal
                    #print('curr_score:'+str(curr_score))
                    # check if moves is empty or if we have new best move(s)
                    if curr_score > best or not moves:
                        best = curr_score
                        alpha = best
                        moves = [(i,j)]
                    # if we have another best move then we add to the moves list
                    elif curr_score == best:
                        moves.append((i,j))

        return moves

        '''
                        # Undo the move
                        board[i][j] = 0

                        # If the value of the current move is
                        # more than the best value, then update
                        # best/
                        if (moveVal > bestVal) :               
                            bestMove = (i, j)
                            bestVal = moveVal

        return bestMove
    '''
    def play(self):
        self.read_file('input.txt')
        start=0
        start_bool=False
        for i in range(5):
            for j in range(5):
                if self.curr_state[i][j]!=0:
                    if i==2 and j==2:
                        start_bool = True
                    start+=1

        if (start==0 and self.color==1) or (start==1 and self.color==2 and start_bool is False):
            action = [(2,2)]
        else:
            action = self.findBestMove( 2, -1000, -1000)
            #print(action)
        # if empty list, then no action, choose to pass
        if action == []:
            rand_action = ['PASS']
        # else choose a random action from the list
        else:
            rand_action = random.choice(action)
        
        print(rand_action)
        self.output(rand_action)

p = myPlayer()
p.play()
#p.evaluate([[2, 0, 1, 1, 0], [0, 0, 2, 1, 0], [0, 0, 2, 0, 0], [0, 2, 0, 1, 0], [0, 0, 0, 0, 0]],1)
#p.read_file('input.txt')
#p.is_alive(3,3,p.curr_state,1)
#p.findBestMove()

#p.minimax(p.prev_state, p.curr_state, 0, False, 0,1000,-1000)

(3, 2)


In [39]:
a=[]
with open('input.txt', 'r') as f:
    list =f.readlines()
b=[[i for i in list[1]]]
c=[]
d=0
if c:
    d=1
d

0

In [2]:
from copy import deepcopy

class myPlayer(object):
    def __init__(self, color=None):
        self.color=color #1 black 2 white
        self.opponent=None
        self.prev_state = []
        self.curr_state=[]
        self.visit=[[0 for i in range(5)] for j in range(5)]
        self.suicide_move=0
        
    def read_file(self, file):
        with open(file, 'r') as f:
            list = f.readlines()
            for i in range(len(list)):
                line=list[i].rstrip('\n')
                if i==0:
                    self.color=int(line)
                    self.opponent=3-self.color
                if i in [1,2,3,4,5]:
                    self.prev_state.append([int(i) for i in line])
                if i in [6,7,8,9,10]:
                    self.curr_state.append([int(i) for i in line])
                    
    def clear_visit(self):
        self.visit =[[0 for i in range(5)] for j in range(5)]
        
    def dfs(self, i, j, board, player): #find liberty
        self.visit[i][j]=1
        directions = [[i - 1, j], [i + 1, j], [i, j - 1], [i, j + 1]]
        for dx, dy in directions:
            if dx<0 or dx>4 or dy<0 or dy>4:
                continue
            elif self.visit[dx][dy]==0:
                if board[dx][dy]==0:
                    return 1
                elif board[dx][dy]==player:
                    self.dfs(dx, dy, board, player)
                else:
                    continue
        return 0
    #1有气 0无气
    
    def is_alive(self, x, y,board,player):

        self.clear_visit()
        isalive =self.dfs(x,y,board,player)            
        self.clear_visit()
        return isalive
    
    def dead_stones(board, player):
        dead =[]
        for i in range(5):
            for j in range(5):
                if board[i][j]==player:
                    if not self.is_alive(i, j, board, player) and (i ,j) not in dead:
                        dead.append((i,j))
                        
        return dead
    
    def capture(self, board, player):
        opponent=3-player
        '''
        capture_list=[]
        
        for i in range(5):
            for j in range(5):
                if board[i][j]==0:
                    continue
                elif board[i][j]==opponent and self.is_alive(i, j, board, opponent)==0:
                    capture_list.append([i,j])
        '''
        capture_list= self.dead_stones(board,player)
        if capture_list:
            for i,j in capture_list:
                board[i][j]=0
        
        for i in range(5):
            for j in range(5):
                if board[i][j]==player:
                    if self.is_alive(i,j,board, player)==0:
                        self.suicide_move=1
                        break
        
        return board
    
    def valid_move_check(self, i, j, player, prev_board, board):
        
        # Check if the place is in the board range
        if not (i >= 0 and i < len(board)):
            return False
        if not (j >= 0 and j < len(board)):
             return False
        
        # Check if the place already has a piece
        if board[i][j] != 0:
             return False
        
        test_board = deepcopy(board)
        prev = deepcopy(prev_board)
        # Check if the place has liberty
        test_board[i][j] = player
        dead_pieces=self.dead_stones(test_board, 3-player)
        if self.is_alive(i, j, test_board, player):
            return True

        # If not, remove the died pieces of opponent and check again
        test_board=self.capture(test_board, 3-player)
        if not self.is_alive(i, j, test_board, player):
            return False

        # Check special case: repeat placement causing the repeat board state (KO rule)
        else:
            if self.is_alive(i,j,test_board, player)>0 and not(dead_pieces and self.KO(prev, test_board)):
                return True
        return False
    
    def KO(self, prev, curr):
        if prev==curr:
            return True
        else:
            return False
    '''    
    def KO(self, i, j, board, player):
        new_board=capture(i,j,board,player)
        if new_board==prev_board:
            return True
        else:
            return False
    '''
        
    def evaluate(self, board, player):
        if player==1:
            eva=0
        else:
            eva=2.5
        for i in board:
            for j in i:
                if j==player:
                    eva+=1 
        return eva           
     
    def game_end(self, board, passs):
        '''
        Check if the game should end.

        :param piece_type: 1('X') or 2('O').
        :param action: "MOVE" or "PASS".
        :return: boolean indicating whether the game should end.
        '''
        n_move=0
        for i in board:
            for j in i:
                if j != 0:
                    n_move+=1
        # Case 1: max move reached
        if n_move>=24:
            return True
        # Case 2: two players all pass the move.
        if passs==2:
            return True
        return False
    
    def minimax(self, prev_board, board, depth, isMax, passs, alpha, beta):
        new_prev =deepcopy(board)
        score = self.evaluate(board, self.color)
        if self.game_end(board, passs) or depth==6:
            return score
        
        if(isMax):
            best =-1000
            valid_moves=0
            for i in range(5):
                for j in range(5):
                    if self.valid_move_check(i, j, self.color, prev_board, board):
                        test_board=deepcopy(board)
                        valid_moves+=1
                        test_board[i][j]=self.color
                        test_board =self.capture(test_board, 3-self.color)
                        '''if prev_board == new_board:
                            continue
                        '''
                        if self.suicide_move==1:
                            self.suicide_move=0
                            best = best
                        else:
                            best = max(best, self.minimax(new_prev, test_board, 
                                                          depth+1, not isMax, passs, alpha, beta))
                            alpha=max(alpha, best)
                            if beta<=alpha:
                                break
                            board[i][j]=0
            if valid_moves==0:
                passs+=1
                best = max(best, self.minimax(new_prev, board, 
                                              depth+1, not isMax, passs, alpha, beta))
            return best
        
        else:
            best = 1000
            valid_moves=0
            for i in range(5):
                for j in range(5):
                    if self.valid_move_check(i, j, self.opponent, prev_board, board):
                        test_board=deepcopy(board)
                        valid_moves+=1
                        test_board[i][j]=self.opponent
                        test_board =self.capture(test_board, 3-self.opponent)
                        '''if prev_board == board:
                            continue
                        '''
                        if self.suicide_move==1:
                            self.suicide_move=0
                            best = best
                        else:
                            best = min(best, self.minimax(new_prev, test_board, 
                                                          depth+1, not isMax, passs, alpha, beta))
                            beta=min(beta, best)
                            if beta<=alpha:
                                break
                            board[i][j]=0
            if valid_moves==0:
                passs+=1
                best = min(best, self.minimax(new_prev, board, 
                                              depth+1, not isMax, passs, alpha, beta))
            return best
    
    def findBestMove(self):
        board =deepcopy(self.curr_state)
        prev =deepcopy(self.prev_state)
        bestVal=-1000
        bestMove=(-1,-1)
        for i in range(5):
            for j in range(5):
                #valid_move_check(self, i, j, player, prev_board, board)
                if self.valid_move_check(i,j,self.color, prev, board) :
                    test_board = deepcopy(board)
                    # Make the move
                    test_board[i][j] = self.color
                    test_board = self.capture(test_board, self.color)
                    '''#KO RULES
                    if board == self.prev_state:
                        continue
                    '''
                    #SUICIDE RULES
                    if self.suicide_move==1:
                        self.suicide_move=0
                        continue
                    else:
                        # compute evaluation function for this
                        # move.
                        moveVal = self.minimax(board,test_board, 0, False, 0, 1000, -1000)

                        # Undo the move
                        board[i][j] = 0

                        # If the value of the current move is
                        # more than the best value, then update
                        # best/
                        if (moveVal > bestVal) :               
                            bestMove = (i, j)
                            bestVal = moveVal

        return bestMove
    
    def output(self):
        self.read_file('input.txt')
        move=self.findBestMove()
        f = open('output.txt','w+')
        if move==(-1,-1):
            f.write('PASS')
            return
        else:
            f.write(str(move[0])+','+str(move[1]))

p = myPlayer()
p.read_file('input.txt')
p.findBestMove()
#p.minimax(p.prev_state, p.curr_state, 0, False, 0,1000,-1000)

(0, 1)

In [ ]:
Class myPlayer:
    def __init__(self, alpha=.7, gamma=.9, initial_value=0.5. color=None):
        if not (0 < gamma <= 1):
            raise ValueError("An MDP must have 0 < gamma <= 1")

        self.color = color
        self.alpha = alpha
        self.gamma = gamma
        self.q_values = {}
        self.prev_state = []
        self.curr_state=[]
        self.initial_value = initial_value
    
    def read_file(self):
        with open(self.file, 'r') as f:
            list = f.readlines()
            for i in range(len(list)):
                line=list[i].rstrip('\n')
                if i==0:
                    self.color=line
                if i in [1,2,3,4,5]:
                    self.prev_state.append()
                if i in [6,7,8,9,10]:
                    self.curr_state.append()
    
    def Q(self, state):
        if state not in self.q_values:
            q_val = np.zeros((5, 5))
            q_val.fill(self.initial_value)
            self.q_values[state] = q_val
        return self.q_values[state]
    
    def _select_best_move(self):
        state=self.curr_state
        q_values =self.Q(state)
        row, col = 0, 0
        curr_max = -np.inf
        while True:
            i, j = self._find_max(q_values)
            if board.is_valid_move(i, j):
                return i, j
            else:
                q_values[i][j] = -1.0
    
    def is_valid_move(i,j):
        
    def _find_max(self, q_values):
        curr_max = -np.inf
        row, col = 0, 0
        for i in range(0, 5):
            for j in range(0, 5):
                if q_values[i][j] > curr_max:
                    curr_max = q_values[i][j]
                    row, col = i, j
        return row, col
    
    def move

In [ ]:
import numpy as np

WIN_REWARD = 1.0
DRAW_REWARD = 0.5
LOSS_REWARD = 0.0

class QLearner:


    GAME_NUM = 100000

    def __init__(self, alpha=.7, gamma=.9, initial_value=0.5, side=None):
        if not (0 < gamma <= 1):
            raise ValueError("An MDP must have 0 < gamma <= 1")

        self.side = side
        self.alpha = alpha
        self.gamma = gamma
        self.q_values = {}
        self.history_states = []
        self.initial_value = initial_value
        # self.state = ?

    def set_side(self, side):
        self.side = side

    def Q(self, state):
        if state not in self.q_values:
            q_val = np.zeros((3, 3))
            q_val.fill(self.initial_value)
            self.q_values[state] = q_val
        return self.q_values[state]

    def _select_best_move(self, board):
        state = board.encode_state()
        q_values = self.Q(state)
        row, col = 0, 0
        curr_max = -np.inf
        while True:
            i, j = self._find_max(q_values)
            if board.is_valid_move(i, j):
                return i, j
            else:
                q_values[i][j] = -1.0

    def _find_max(self, q_values):
        curr_max = -np.inf
        row, col = 0, 0
        for i in range(0, 3):
            for j in range(0, 3):
                if q_values[i][j] > curr_max:
                    curr_max = q_values[i][j]
                    row, col = i, j
        return row, col


    def move(self, board):
        """ make a move
        """
        if board.game_over():
            return
        row, col = self._select_best_move(board)
        self.history_states.append((board.encode_state(), (row, col)))
        return board.move(row, col, self.side)

    def learn(self, board):
        """ when games ended, this method will be called to update the qvalues
        """
        if board.game_result == 0:
            reward = DRAW_REWARD
        elif board.game_result == self.side:
            reward = WIN_REWARD
        else:
            reward = LOSS_REWARD
        self.history_states.reverse()
        max_q_value = -1.0
        for hist in self.history_states:
            state, move = hist
            q = self.Q(state)
            if max_q_value < 0:
                q[move[0]][move[1]] = reward
            else:
                q[move[0]][move[1]] = q[move[0]][move[1]] * (1 - self.alpha) + self.alpha * self.gamma * max_q_value
            max_q_value = np.max(q)
        self.history_states = []
